In [1]:
# Importing necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
# Import and read the charity_data.csv
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column
unique_values = application_df.nunique()
unique_values

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Binning 'APPLICATION_TYPE' column
cutoff = 500
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(app_type_counts[app_type_counts < cutoff].index)
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

In [6]:
# Binning 'CLASSIFICATION' column
class_cutoff = 1000
class_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(class_counts[class_counts < class_cutoff].index)
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

In [7]:
# Encoding categorical variables
application_encoded = pd.get_dummies(application_df)

In [8]:
# Splitting the data into features and target
X = application_encoded.drop(columns=["IS_SUCCESSFUL"])
y = application_encoded["IS_SUCCESSFUL"]

In [9]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [10]:
# Standardizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 256
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 64

nn = tf.keras.models.Sequential()

In [12]:
# Adding the input layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Adding dropout regularization
nn.add(tf.keras.layers.Dropout(0.3))

# Adding the second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Adding dropout regularization
nn.add(tf.keras.layers.Dropout(0.3))

# Adding the third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Adding the output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compiling the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Define a checkpoint callback
checkpoint_filepath = 'best_model.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [14]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=250, verbose=1, validation_split=0.2, callbacks=[model_checkpoint_callback])

Epoch 1/250
644/644 [==============================] - 1s 918us/step - loss: 0.5786 - accuracy: 0.7202 - val_loss: 0.5638 - val_accuracy: 0.7254
Epoch 2/250
644/644 [==============================] - 1s 818us/step - loss: 0.5613 - accuracy: 0.7288 - val_loss: 0.5686 - val_accuracy: 0.7242
Epoch 3/250
644/644 [==============================] - 1s 915us/step - loss: 0.5588 - accuracy: 0.7323 - val_loss: 0.5613 - val_accuracy: 0.7285
Epoch 4/250
644/644 [==============================] - 1s 832us/step - loss: 0.5559 - accuracy: 0.7318 - val_loss: 0.5602 - val_accuracy: 0.7259
Epoch 5/250
644/644 [==============================] - 1s 805us/step - loss: 0.5537 - accuracy: 0.7333 - val_loss: 0.5641 - val_accuracy: 0.7238
Epoch 6/250
644/644 [==============================] - 1s 825us/step - loss: 0.5526 - accuracy: 0.7331 - val_loss: 0.5629 - val_accuracy: 0.7219
Epoch 7/250
644/644 [==============================] - 1s 814us/step - loss: 0.5526 - accuracy: 0.7339 - val_loss: 0.5632 - val_ac

644/644 [==============================] - 1s 831us/step - loss: 0.5395 - accuracy: 0.7399 - val_loss: 0.5602 - val_accuracy: 0.7254
Epoch 58/250
644/644 [==============================] - 1s 820us/step - loss: 0.5407 - accuracy: 0.7394 - val_loss: 0.5628 - val_accuracy: 0.7252
Epoch 59/250
644/644 [==============================] - 1s 821us/step - loss: 0.5408 - accuracy: 0.7396 - val_loss: 0.5588 - val_accuracy: 0.7246
Epoch 60/250
644/644 [==============================] - 1s 832us/step - loss: 0.5385 - accuracy: 0.7400 - val_loss: 0.5606 - val_accuracy: 0.7217
Epoch 61/250
644/644 [==============================] - 1s 842us/step - loss: 0.5393 - accuracy: 0.7403 - val_loss: 0.5592 - val_accuracy: 0.7256
Epoch 62/250
644/644 [==============================] - 1s 844us/step - loss: 0.5400 - accuracy: 0.7400 - val_loss: 0.5594 - val_accuracy: 0.7248
Epoch 63/250
644/644 [==============================] - 1s 843us/step - loss: 0.5393 - accuracy: 0.7409 - val_loss: 0.5593 - val_accuracy

644/644 [==============================] - 1s 784us/step - loss: 0.5355 - accuracy: 0.7425 - val_loss: 0.5831 - val_accuracy: 0.7252
Epoch 114/250
644/644 [==============================] - 1s 784us/step - loss: 0.5362 - accuracy: 0.7414 - val_loss: 0.5806 - val_accuracy: 0.7252
Epoch 115/250
644/644 [==============================] - 1s 805us/step - loss: 0.5372 - accuracy: 0.7408 - val_loss: 0.5845 - val_accuracy: 0.7190
Epoch 116/250
644/644 [==============================] - 1s 788us/step - loss: 0.5353 - accuracy: 0.7421 - val_loss: 0.5840 - val_accuracy: 0.7205
Epoch 117/250
644/644 [==============================] - 1s 786us/step - loss: 0.5361 - accuracy: 0.7416 - val_loss: 0.5818 - val_accuracy: 0.7250
Epoch 118/250
644/644 [==============================] - 1s 787us/step - loss: 0.5364 - accuracy: 0.7423 - val_loss: 0.5873 - val_accuracy: 0.7259
Epoch 119/250
644/644 [==============================] - 1s 814us/step - loss: 0.5364 - accuracy: 0.7416 - val_loss: 0.5905 - val_ac

Epoch 169/250
644/644 [==============================] - 1s 798us/step - loss: 0.5352 - accuracy: 0.7426 - val_loss: 0.5958 - val_accuracy: 0.7244
Epoch 170/250
644/644 [==============================] - 1s 801us/step - loss: 0.5336 - accuracy: 0.7424 - val_loss: 0.5955 - val_accuracy: 0.7228
Epoch 171/250
644/644 [==============================] - 1s 795us/step - loss: 0.5350 - accuracy: 0.7407 - val_loss: 0.6067 - val_accuracy: 0.7236
Epoch 172/250
644/644 [==============================] - 1s 799us/step - loss: 0.5353 - accuracy: 0.7426 - val_loss: 0.6241 - val_accuracy: 0.7199
Epoch 173/250
644/644 [==============================] - 1s 794us/step - loss: 0.5343 - accuracy: 0.7422 - val_loss: 0.6161 - val_accuracy: 0.7236
Epoch 174/250
644/644 [==============================] - 1s 795us/step - loss: 0.5348 - accuracy: 0.7406 - val_loss: 0.6003 - val_accuracy: 0.7226
Epoch 175/250
644/644 [==============================] - 1s 795us/step - loss: 0.5347 - accuracy: 0.7422 - val_loss: 0

Epoch 225/250
644/644 [==============================] - 1s 795us/step - loss: 0.5346 - accuracy: 0.7425 - val_loss: 0.6186 - val_accuracy: 0.7250
Epoch 226/250
644/644 [==============================] - 1s 785us/step - loss: 0.5336 - accuracy: 0.7428 - val_loss: 0.6611 - val_accuracy: 0.7195
Epoch 227/250
644/644 [==============================] - 1s 791us/step - loss: 0.5346 - accuracy: 0.7428 - val_loss: 0.6393 - val_accuracy: 0.7238
Epoch 228/250
644/644 [==============================] - 1s 795us/step - loss: 0.5340 - accuracy: 0.7422 - val_loss: 0.6310 - val_accuracy: 0.7205
Epoch 229/250
644/644 [==============================] - 1s 797us/step - loss: 0.5341 - accuracy: 0.7417 - val_loss: 0.6257 - val_accuracy: 0.7207
Epoch 230/250
644/644 [==============================] - 1s 794us/step - loss: 0.5337 - accuracy: 0.7434 - val_loss: 0.6047 - val_accuracy: 0.7261
Epoch 231/250
644/644 [==============================] - 1s 799us/step - loss: 0.5340 - accuracy: 0.7423 - val_loss: 0

In [15]:
# Load the best model
nn.load_weights(checkpoint_filepath)

In [16]:
# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5561 - accuracy: 0.7265 - 80ms/epoch - 298us/step
Loss: 0.5560771226882935, Accuracy: 0.7265306115150452


In [17]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

/Users/mira.smith/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
